In [1]:
import os
# by default notebook runs in research folder
os.chdir("../../")
%pwd

'/Users/ashleyalexjacob/Data Science/Chicken-Disease-Classifer'

In [2]:
import tensorflow as tf

2023-06-20 01:58:22.668679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model  = tf.keras.models.load_model('artifacts/training/model.h5')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    trained_model_path: Path
    training_data_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from src.constants import *
from src.utils.commons import read_yaml, create_directories, save_json


In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> ModelEvaluationConfig:
        evaluation_config  = ModelEvaluationConfig(
            trained_model_path = 'artifacts/training/model.h5',
            training_data_path = 'artifacts/data_ingestion/Chicken-fecal-images',
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE, 
        )
        return evaluation_config

In [7]:
from urllib.parse import urlparse

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) :
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30,

        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset='validation',
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluate_model(self):
        self.model = self.load_model(self.config.trained_model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_scores(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path= Path("scores.json"), data = scores)
        

In [10]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation  = ModelEvaluation(evaluation_config)
    evaluation.evaluate_model()
    evaluation.save_scores()

except Exception as e:
    raise e

[2023-06-20 02:25:19,772: INFO: commons: Yaml File: config/config.yaml Loaded]
[2023-06-20 02:25:19,779: INFO: commons: Yaml File: params.yaml Loaded]
[2023-06-20 02:25:19,783: INFO: commons: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 19s 2s/step - loss: 17.7990 - accuracy: 0.5000
[2023-06-20 02:25:40,102: INFO: commons: json file saved at: scores.json]
